# Entrainement sur les données moléculaires

Dans ce notebook on va tenter d'entrainer des modeles eseulement avec les données moléculaires pour créer des features additionnels pour un meta model ensuite, on va voir si on peut predire la proba d'une mort = 1 avec ces donnés et voir aussi la tache de regression sur le temps de survie.